In [ ]:
from model.ctabgan import CTABGAN
from model.eval.evaluation import get_utility_metrics,stat_sim,privacy_metrics
import numpy as np
import pandas as pd
import glob

In [3]:
import os
pid = os.getpid()
print(f"Kernel PID: {pid}")

Kernel PID: 43962


In [2]:
num_exp = 1
# dataset = "Adult"
# real_path = "Real_Datasets/Adult.csv"
# fake_file_root = "Fake_Datasets"

dataset = "real_support2_small_Ostar_DP"
real_path = "Real_Datasets/real_support2_small_Ostar.csv"
fake_file_root = "Fake_Datasets"

In [3]:
# synthesizer =  CTABGAN(raw_csv_path = real_path,
#                  test_ratio = 0.20,
#                  categorical_columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country', 'income'], 
#                  log_columns = [],
#                  mixed_columns= {'capital-loss':[0.0],'capital-gain':[0.0]},
#                  general_columns = ["age"],
#                  non_categorical_columns = [],
#                  integer_columns = ['age', 'fnlwgt','capital-gain', 'capital-loss','hours-per-week'],
#                  problem_type= {"Classification": 'income'}) 

synthesizer =  CTABGAN(raw_csv_path = real_path,
                 test_ratio = 0.20,
                 categorical_columns = ['Y'], # any categorical column (binary, multi class)
                 log_columns = ['bun','crea', 'bili', 'wblc', 'dnrday', 'hday', 'totmcst', 'totcst', 'charges', 'slos'], # long tailed ditributions: assessed by empirical marginal histograms
                 mixed_columns= {}, # categorical and continuous data in one feature
                 general_columns = [], # only for simple, single mode continuous distributions or for categorical features that have computationally prohibitive many categories
                 non_categorical_columns = [], # no idea what this is
                 integer_columns = [],
                 problem_type= {"Classification": 'Y'}) 

for i in range(num_exp):
    synthesizer.fit()
    syn = synthesizer.generate_samples()
    # syn.to_csv(fake_file_root+"/"+dataset+"/"+ dataset+"_fake_{exp}.csv".format(exp=i), index= False)
    syn.to_csv(f'{fake_file_root}/{dataset}/{dataset}_fake_{i}.csv', index= False)

  0%|          | 0/150 [00:00<?, ?it/s]

: 

In [12]:
fake_paths = glob.glob(fake_file_root+"/"+dataset+"/"+"*")

In [13]:
classifiers_list = ["lr","dt","rf","mlp"]
result_mat = get_utility_metrics(real_path,fake_paths,"MinMax",classifiers_list, test_ratio = 0.20)

result_df  = pd.DataFrame(result_mat,columns=["Acc","AUC","F1_Score"])
result_df.index = classifiers_list
result_df

,Acc,AUC,F1_Score
lr,9.830508,0.127107,0.236366
dt,13.898305,0.118599,0.127792
rf,12.881356,0.172128,0.225515
mlp,8.135593,0.131777,0.198161


In [ ]:
adult_categorical = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country', 'income']
stat_res_avg = []
for fake_path in fake_paths:
    stat_res = stat_sim(real_path,fake_path,adult_categorical)
    stat_res_avg.append(stat_res)

stat_columns = ["Average WD (Continuous Columns","Average JSD (Categorical Columns)","Correlation Distance"]
stat_results = pd.DataFrame(np.array(stat_res_avg).mean(axis=0).reshape(1,3),columns=stat_columns)
stat_results

In [ ]:
priv_res_avg = []
for fake_path in fake_paths:
    priv_res = privacy_metrics(real_path,fake_path)
    priv_res_avg.append(priv_res)
    
privacy_columns = ["DCR between Real and Fake (5th perc)","DCR within Real(5th perc)","DCR within Fake (5th perc)","NNDR between Real and Fake (5th perc)","NNDR within Real (5th perc)","NNDR within Fake (5th perc)"]
privacy_results = pd.DataFrame(np.array(priv_res_avg).mean(axis=0).reshape(1,6),columns=privacy_columns)
privacy_results